# Proyecto Showz

## Introduccion 
Para el desarrollo de este proyecto, buscamos optimizar los gastos de marketing basando la investigación en el análisis de datos de comportamiento de los usuarios. A su vez, se establecerán cohortes de clientes con el objetivo de entender cómo, cuándo y cuánto aportan a la compañía. Este enfoque nos permitirá identificar patrones de uso, momentos clave en el ciclo de vida del cliente y calcular el retorno de la inversión en adquisición, facilitando así una mejor toma de decisiones estratégicas.

Contamos con los registros del servidor con los siguientes datasets:
- visits.csv: visitas al sitio (ID, fuente, timestamp, etc.).

- orders.csv: pedidos (cliente, fecha, importe, etc.).

- costs.csv: gastos de marketing (fecha, canal, inversión).

# Objetivo General 
``Optimizar los gastos de marketing entendiendo cómo, cuándo y cuánto aportan los clientes.``

## Descripcion de los datos
La tabla `visits` (registros del servidor con datos sobre las visitas al sitio web):

- Uid: identificador único del usuario.
- Device: dispositivo del usuario.
- Start Ts: fecha y hora de inicio de la sesión.
- End Ts: fecha y hora de término de la sesión.
- Source Id: identificador de la fuente de anuncios de la que proviene el usuario.

Todas las fechas de esta tabla están en formato AAAA-MM-DD.

La tabla `orders` (datos sobre pedidos):

- Uid: identificador único del usuario que realiza un pedido.
- Buy Ts: fecha y hora del pedido. Revenue: el ingreso de Showz por el pedido.

La tabla `costs` (datos sobre gastos de marketing):

- source_id: identificador de la fuente de anuncios.
- dt: fecha.
- costs: gastos en esta fuente de anuncios en este día.

In [75]:
# Cargar todas las librerias
import pandas as pd 


In [76]:
# Designar las variables de los Dataframe 
visits= pd.read_csv('/Users/ASUS/OneDrive/Documentos/Tripleten/Proyecto Spring 9/visits_log_us.csv', sep=',') 
orders=pd.read_csv('/Users/ASUS/OneDrive/Documentos/Tripleten/Proyecto Spring 9/orders_log_us.csv', sep=',')
costs=pd.read_csv('/Users/ASUS/OneDrive/Documentos/Tripleten/Proyecto Spring 9/costs_us.csv', sep=',')

## Nombres asignados a los DataFrames

Para garantizar un desarrollo estructurado y eficiente del proyecto, se asignaron nombres descriptivos a los *DataFrames* generados a partir de las consultas a los servidores. Esta práctica permite mantener un flujo de trabajo organizado y facilita la interpretación de los resultados en las distintas etapas del análisis. Las asignaciones realizadas son las siguientes:

- `visits`: corresponde al resultado de la consulta `visits_log_us`.
- `orders`: corresponde al resultado de la consulta `orders_log_us`.
- `costs`: corresponde al resultado de la consulta `costs_us`.

A continuación, el análisis exploratorio de los datos.


In [77]:
# Visualizacion preliminar de los dataframes:

visits.info()

# estandarizar datos:
for col in visits.select_dtypes(include='object'):
    visits[col] = visits[col].str.lower()

print('\n', visits.sample(n=5))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 359400 entries, 0 to 359399
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   Device     359400 non-null  object
 1   End Ts     359400 non-null  object
 2   Source Id  359400 non-null  int64 
 3   Start Ts   359400 non-null  object
 4   Uid        359400 non-null  uint64
dtypes: int64(1), object(3), uint64(1)
memory usage: 13.7+ MB

          Device               End Ts  Source Id             Start Ts  \
156178  desktop  2017-11-28 12:57:00          2  2017-11-28 12:53:00   
83292   desktop  2017-12-05 13:17:00          3  2017-12-05 13:14:00   
225053    touch  2018-02-04 22:28:00          4  2018-02-04 22:27:00   
69491   desktop  2017-12-12 17:52:00          4  2017-12-12 17:51:00   
55140   desktop  2018-03-28 23:16:00          5  2018-03-28 22:58:00   

                         Uid  
156178  18069293404286467993  
83292   15574612294865373014  
225053   61350292574

In [78]:
# Visualizacion preliminar de los dataframes:

orders.info()

print('\n', orders.sample(n=5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50415 entries, 0 to 50414
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Buy Ts   50415 non-null  object 
 1   Revenue  50415 non-null  float64
 2   Uid      50415 non-null  uint64 
dtypes: float64(1), object(1), uint64(1)
memory usage: 1.2+ MB

                     Buy Ts  Revenue                   Uid
5644   2017-08-19 18:48:00     1.33   9103966622588051356
24257  2017-12-15 15:22:00     2.44  12917425213294036126
18136  2017-11-17 19:10:00     3.30  11920452646463905188
14146  2017-10-23 12:14:00     1.83   2276755474897127212
7337   2017-09-09 20:46:00     0.92   6747682981491646964


In [79]:
# Visualizacion preliminar de los dataframes:

costs.info()

print('\n', costs.sample(n=5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2542 entries, 0 to 2541
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   source_id  2542 non-null   int64  
 1   dt         2542 non-null   object 
 2   costs      2542 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 59.7+ KB

       source_id          dt   costs
2409         10  2018-01-17   23.92
2257         10  2017-08-18   16.98
2016          9  2017-12-18   23.15
2369         10  2017-12-08    9.50
801           3  2017-08-15  192.73


## Correcion de tipo de datos

Dado que la naturaleza de los datos requiere un procesamiento adecuado, es fundamental contar con una estructura organizada y coherente. Por esta razón, se ha llevado a cabo una reorganización estratégica de los conjuntos de datos, lo cual permitirá facilitar su análisis en cada etapa del proyecto. A continuación, se detallan las transformaciones realizadas específicamente en los dataframes visits, orders y costs:

- En ``visits``, se renombran las columnas para eliminar espacios y facilitar el manejo del código (Start Ts a start_ts, End Ts a end_ts, etc.), y se convierten las columnas de fechas al formato datetime.

- En ``orders``, se transforma la columna Buy Ts a buy_ts y se asegura que los ingresos (revenue) estén en el tipo numérico correcto.

- En ``costs``, se homogeneizan los nombres (dt para la fecha, costs para los gastos) y se convierte la columna de fechas a datetime.

Esta preparación garantiza que los datos estén listos para los análisis de comportamiento, cohortes y rentabilidad que se desarrollarán a lo largo del proyecto.